### 모델 매개변수 최적화

- 모델을 학습하는 과정
    - 각 반복 단계에서 모델은 출력을 추측
    - 추측과 정답 사이의 오류(손실, loss) 계산
    - 매개변수에 대한 loss의 도함수(derivate) 수집
    - 경사하강법을 사용하여 이 파라미터들을 최적화(optimize)


#### 기본 (Pre-requisite) 코드

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

#### 하이퍼파라미터(Hyperparameter)
- Hyperparameter: 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
- 학습 시에는 다음과 같은 하이퍼파라미터 정의
    - 에폭(epoch)수: 데이터셋을 반복하는 횟수
    - 배치 크기(batch size): 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
    - 학습률(learning rate): 각 매치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있음

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

#### 최적화 단계 (Optimization Loop)
- 하이퍼파라미터 설정 이후 최적화 단계를 통해 모델을 학습하고 최적화
- 최적화 단계의 각 반복(iteration)을 `epoch`이라고 함
- 하나의 에폭은 다음 두 부분으로 구성
    - 학습 단계(train loop) : 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴
    - 검증/테스트 단계(validation/test loop) : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iteration)

#### 손실 함수(loss function)
- loss function: 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정
- 학습 중에 이 값을 최소화하려고 함
- label과 prediction을 비교하여 손실(loss) 계산
- 종류
    - `nn.MSELoss`(평균 제곱 오차(MSE: Mean Square Error)) : regression task에 사용
    - `nn.NLLLoss`(음의 로그 우도(Negative Log Likelihood)): classification task에 사용
    - `nn.CrossEntropyLoss`: `nn.LogSoftmax`와 `nn.NLLLoss`를 합친 손실함수

In [3]:
loss_fn = nn.CrossEntropyLoss()

#### 옵티마이저(Optimizer)
- 최적화 : 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정
- 최적화 알고리즘 : 이 과정이 수행되는 방식 (여기에서는 `확률적 경사하강법(SGD: Stochastic Gradient Descent)` 정의)
- 모든 최적화 절차(logic)은 `optimizer`객체에 캡슐화(encapsulate)

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

학습 단계(loop)에서 최적화는 세 단계로 이루어짐
- `optimizer.zero_grad()`를 호출하여 모델 매개변수의 변화도를 재설정. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
- `loss.backwards()`를 호출하여 예측 손실(prediction loss)을 역전파. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장
- 변화도를 계산한 뒤에는 `optimizer.step()`을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정

#### 전체 구현

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307240  [   64/60000]
loss: 2.297465  [ 6464/60000]
loss: 2.284309  [12864/60000]
loss: 2.279191  [19264/60000]
loss: 2.265481  [25664/60000]
loss: 2.245048  [32064/60000]
loss: 2.239249  [38464/60000]
loss: 2.215512  [44864/60000]
loss: 2.212602  [51264/60000]
loss: 2.181991  [57664/60000]
Test Error: 
 Accuracy: 47.9%, Avg loss: 2.178944 

Epoch 2
-------------------------------
loss: 2.182241  [   64/60000]
loss: 2.174576  [ 6464/60000]
loss: 2.131202  [12864/60000]
loss: 2.144742  [19264/60000]
loss: 2.095996  [25664/60000]
loss: 2.055883  [32064/60000]
loss: 2.064225  [38464/60000]
loss: 2.003603  [44864/60000]
loss: 2.003593  [51264/60000]
loss: 1.930675  [57664/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.934527 

Epoch 3
-------------------------------
loss: 1.958361  [   64/60000]
loss: 1.934004  [ 6464/60000]
loss: 1.835813  [12864/60000]
loss: 1.865505  [19264/60000]
loss: 1.757957  [25664/60000]
loss: 1.719600  [32064/600